In [1]:
import pandas as pd
import json
import os
from typing import Dict, Any, Optional, Tuple
from datetime import datetime, timedelta
from sub_func import *

In [2]:
def extract_key_metrics(sector_info):
    """섹터 정보에서 Carhart 4 factor 관련 주요 지표 추출"""
    if sector_info is None:
        return None
    
    return {
        'market_beta': sector_info.get('market_beta', 0),
        'size_factor': sector_info.get('size_factor', 0),
        'value_factor': sector_info.get('value_factor', 0),
        'momentum_factor': sector_info.get('momentum_factor', 0)
    }

def extract_sentiment_score(df):
    """감성분석 결과를 numerical score로 변환"""
    def get_score(result):
        if isinstance(result, dict):
            # 딕셔너리에서 감성 결과 추출 (예: result.get('sentiment') 등)
            sentiment = result.get('sentiment', 'neutral')  # 적절한 키로 수정
        else:
            sentiment = result
            
        score_mapping = {
            'positive': 1.0,
            'neutral': 0.0,
            'negative': -1.0
        }
        return score_mapping.get(sentiment, 0.0)
    
    df['sentiment_score'] = df['SA_result'].apply(get_score)
    return df

def filter_by_percentile_and_label(df, label, percentile):
    """특정 감성의 상위/하위 percentile에 해당하는 뉴스 필터링"""
    if df.empty:
        return pd.DataFrame()
    
    # label에 따라 필터링
    if label == 'positive':
        filtered_df = df[df['SA_result'] == 'positive']
        return filtered_df.nlargest(int(len(filtered_df) * percentile/100), 'sentiment_score')
    else:  # negative
        filtered_df = df[df['SA_result'] == 'negative']
        return filtered_df.nsmallest(int(len(filtered_df) * percentile/100), 'sentiment_score')

In [3]:
class InvestmentReportGenerator:
    """Investment report generation class with GPT integration"""
    
    MARKDOWN_INSTRUCTION = """
    응답은 반드시 markdown 문법에 따라 작성되어야 합니다.
    ** 보고서에는 반드시 주어진 정보에 대한 분석이 필요합니다 **
    """

    ANALYST_BASE_PROMPT = """
    당신은 증권회사에 고용된 {role}입니다.
    주식투자의 관점에서 주어진 정보들을 요약하고, 이에 대한 의견을 알려주세요.
    {additional_instructions}
    {markdown_instruction}
    """

    INDIVIDUAL_REPORT_SYSTEM = """증권사 애널리스트로서 종목 분석 보고서 작성
        # 필수 섹션
        1. 기업 개요
        - 사업 모델과 핵심 역량
        - 시장 포지셔닝

        2. 재무 분석
        - 핵심 재무지표 분석
        - 수익성/성장성 평가

        3. 섹터 분석
        - 산업 동향과 경쟁력
        - 기술적 분석 시사점

        4. 투자의견
        - 투자포인트 3개
        - 주요 리스크
        - 목표가 및 근거

        요구사항:
        - 구체적 데이터 기반
        - 명확한 투자 논리 제시
        """ + MARKDOWN_INSTRUCTION

    def __init__(self, tickers: list, year: str, quarter: str):
        self.tickers = tickers
        self.year = year
        self.quarter = quarter
        self.prompts = self._initialize_prompts()
        self.responses = {ticker: {} for ticker in tickers}
        self.individual_reports = {}
        self.start_date, self.end_date = self._get_date_range()
        
    def generate_individual_report(self, ticker: str) -> str:
        """Generate a comprehensive report for a single stock"""
        print(f"\n=== {ticker} 분석 중... ===")
        
        try:
            # Financial analysis
            financial_data = self.analyze_financial_data(ticker)
            
            # Sector and pattern analysis
            sector_analysis = self.analyze_sector_and_pattern(ticker)
            
            # News analysis with enhanced error handling
            try:
                news_data = corp_rel_news_info(ticker, self.year, self.start_date, self.end_date)
                news_summary = self._process_news(news_data) if news_data is not None else {"Positive": [], "Negative": []}
            except FileNotFoundError:
                print(f"{ticker}의 뉴스 데이터가 없습니다. 분석을 계속합니다.")
                news_summary = {"Positive": [], "Negative": []}
            except Exception as e:
                print(f"{ticker}의 뉴스 처리 중 오류 발생: {e}. 분석을 계속합니다.")
                news_summary = {"Positive": [], "Negative": []}
            
            # Stock price data
            try:
                stock_price = stock_price_info(ticker, self.start_date, self.end_date)
                price_dict = stock_price.to_dict() if stock_price is not None and not stock_price.empty else None
            except Exception as e:
                print(f"{ticker}의 주가 정보 처리 중 오류 발생: {e}. 분석을 계속합니다.")
                price_dict = None
            
            # Combine all data for individual report
            report_prompt = "\n".join([
                f"=== {ticker} 종목 분석 ===",
                f"재무제표 및 재무 비율 분석: {financial_data}",
                f"섹터 분석: {sector_analysis}",
                f"종목 관련 뉴스: {news_summary}",
                f"주가 정보: {price_dict}"
            ])
            
            # Generate individual report
            report = to_GPT(self.INDIVIDUAL_REPORT_SYSTEM, report_prompt)
            self.individual_reports[ticker] = report
            return report
            
        except Exception as e:
            print(f"{ticker} 분석 중 오류 발생: {e}")
            return None
    
    def _initialize_prompts(self) -> Dict[str, str]:
        """Initialize system prompts with templated format"""
        return {
            "financial_system": self.ANALYST_BASE_PROMPT.format(
                role="재무전문가",
                additional_instructions="보고서 근거 기반 의견 제시",
                markdown_instruction=self.MARKDOWN_INSTRUCTION
            ),
            "intl_macro_system": self.ANALYST_BASE_PROMPT.format(
                role="국제관계전문가",
                additional_instructions="국가별 금리, GDP, 인플레이션 등 거시경제 정보 분석",
                markdown_instruction=self.MARKDOWN_INSTRUCTION
            ),
            "sector_system": """증권사 경제전문가로서 투자 관점에서 정보 분석 및 의견 제시
                # 필수 포함 사항
                - 섹터별 성과와 동향 분석
                - 투자 매력도 평가 (근거 제시)
                - 차트 패턴 분석 및 기술적 시사점
                """ + self.MARKDOWN_INSTRUCTION,
            "final_system": """증권사 리서치센터장으로서 개별 애널리스트 보고서들을 종합하여 최종 투자전략 보고서 작성
                # 필수 섹션
                1. 거시경제 분석 요약
                - 글로벌 동향 핵심 포인트
                - 주요 리스크 요인

                2. 개별 종목 분석 종합
                - 각 종목 투자매력도 비교
                - 상대가치 평가

                3. 최종 포트폴리오 전략
                - 종목별 투자비중 추천과 근거
                - 위험관리 방안

                4. 핵심 결론
                - 최우선 투자 추천 종목
                - 중점 모니터링 요소

                작성 지침:
                - 개별 애널리스트 보고서의 분석을 비교/종합하여 결론 도출
                - 종목간 상대매력도를 구체적 근거와 함께 제시
                - 실행 가능한 투자전략 제안
                """ + self.MARKDOWN_INSTRUCTION
        }

    def _get_date_range(self) -> tuple:
        """Get start and end dates for the given quarter"""
        quarter_months = {
            'Q1': ('01', '03'),
            'Q2': ('04', '06'),
            'Q3': ('07', '09'),
            'Q4': ('10', '12')
        }
        
        if self.quarter in quarter_months:
            start_month, end_month = quarter_months[self.quarter]
            start_date = f"{self.year}{start_month}01"
            end_date = f"{self.year}{end_month}{'30' if end_month in ['06', '09'] else '31'}"
            return start_date, end_date
        else:
            raise ValueError(f"Invalid quarter: {self.quarter}")

    def analyze_financial_data(self, ticker: str) -> str:
        """Analyze financial statements and generate report"""
        try:
            fin_statement = get_raw_fin_statement_info(ticker, self.year, self.quarter)
            fin_statement_dict = fin_statement.to_dict() if fin_statement is not None else {}
        except Exception:
            fin_statement_dict = {}

        try:
            fin_ratio = fin_statement_info(ticker, self.year, self.quarter)
            fin_ratio_dict = fin_ratio.to_dict('records')[0] if fin_ratio is not None and not fin_ratio.empty else {}
        except Exception:
            fin_ratio_dict = {}

        try:
            fin_report = reports_info(ticker, self.year, self.quarter)
            report_content = fin_report['1. 요약재무정보.csv'][0][4:-4] if not fin_report.empty else "정보 없음"
        except Exception:
            report_content = "정보 없음"
        
        prompt_data = {
            "재무제표": fin_statement_dict,
            "주요 재무 비율": fin_ratio_dict,
            "재무보고서": report_content
        }
        
        self.prompts["financial_prompt"] = "\n".join(f"{k}: {v}" for k, v in prompt_data.items())
        return to_GPT(self.prompts["financial_system"], self.prompts["financial_prompt"])

    def analyze_international_macro(self) -> str:
        """Analyze international news and macroeconomic data"""
        try:
            intl_news = intl_news_info(self.year, self.start_date, self.end_date)
            news_titles = list(intl_news['news_title']) if intl_news is not None and not intl_news.empty else []
        except Exception:
            news_titles = []

        try:
            macro_data = macro_econ_info(self.year, self.start_date, self.end_date)
        except Exception:
            macro_data = "거시경제 데이터 없음"
        
        self.prompts["intl_macro_prompt"] = "\n".join([
            f"국제 뉴스 헤드라인: {news_titles}",
            f"거시경제 관련 정보: {macro_data}"
        ])
        
        return to_GPT(self.prompts["intl_macro_system"], self.prompts["intl_macro_prompt"])

    def analyze_sector_and_pattern(self, ticker: str) -> str:
        """Analyze sector trends and chart patterns"""
        index_prices = {}
        try:
            sector_list = [s for s in os.listdir('../store_data/raw/market_data/sector') 
                          if '코스피' not in s]
        except Exception:
            sector_list = []
        
        # Collect sector data
        for sector in sector_list:
            try:
                index_price = index_price_info(sector, self.start_date, self.end_date)
                if index_price is not None and not index_price.empty:
                    index_price = index_price[['Close', 'Transaction_Val', 'Market_Cap', 'RSI_14']]
                    index_prices[sector] = index_price.T.to_dict()
            except Exception:
                continue
        
        # Collect sector analysis
        sector_infos = {}
        for sector in sector_list:
            try:
                sector_analysis = sector_analysis_info(sector, self.year, self.quarter)
                if sector_analysis is not None:
                    sector_infos[sector] = extract_key_metrics(sector_analysis)
            except Exception:
                continue

        try:
            pattern_data = pattern_info(ticker, self.end_date.replace('-', ''))
            pattern_dict = pattern_data.to_dict('records') if pattern_data is not None and not pattern_data.empty else None
        except Exception:
            pattern_dict = None
        
        self.prompts["sector_prompt"] = "\n".join([
            f"섹터별 가격 정보: {index_prices}",
            f"섹터별 carhart 4 factor 분석: {sector_infos}",
            f"차트 패턴 분석 결과: {pattern_dict}"
        ])
        
        return to_GPT(self.prompts["sector_system"], self.prompts["sector_prompt"])

    def analyze_stocks(self):
        """Execute analysis for all stocks"""
        # Only analyze macro once
        macro_response = self.analyze_international_macro()
        self.responses["international_macro"] = macro_response
        
        for ticker in self.tickers:
            print(f"\n=== {ticker} 분석 중... ===")
            try:
                self.responses[ticker].update({
                    "financial": self.analyze_financial_data(ticker),
                    "sector_pattern_analysis": self.analyze_sector_and_pattern(ticker)
                })
            except Exception as e:
                print(f"{ticker} 분석 중 오류 발생: {e}")

    def generate_final_report(self) -> dict:
        try:
            # 종목들의 개별 보고서만 포함하기
            combined_prompt = []
            
            for ticker in self.tickers:
                if ticker not in self.individual_reports:
                    print(f"{ticker} 보고서 없음")
                    pass
                
                report = self.individual_reports.get(ticker)
                if report:
                    combined_prompt.append(f"\n=== {ticker} 종목 분석 ===")
                    report_content = report.get('choices', [{}])[0].get('message', {}).get('content', '') if isinstance(report, dict) else str(report)
                    combined_prompt.append(report_content)

            prompt = "\n".join(combined_prompt)

            # 최종 보고서 생성 요청
            final_response = to_GPT(self.prompts["final_system"], prompt)
            return final_response
        
        except Exception as e:
            print(f"generate_final_report에서 예외 발생: {e}")
            return {}

    
    def _process_news(self, corp_news_df) -> Dict[str, list]:
        """Process corporate news and extract sentiment"""
        news_summary = {'Positive': [], 'Negative': []}
        
        if corp_news_df is not None and not corp_news_df.empty:
            try:
                # 증권 카테고리 필터링
                corp_news_df = corp_news_df[corp_news_df['news_category'].str.contains('증권', na=False)]
                
                if not corp_news_df.empty:
                    # SA 결과 및 감성 점수 추출
                    corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x: 
                        get_SA_result(x) if pd.notna(x) else None)
                    
                    # None이나 NaN이 아닌 행만 감성 점수 추출
                    valid_news = corp_news_df.dropna(subset=['SA_result'])
                    if not valid_news.empty:
                        valid_news = extract_sentiment_score(valid_news)
                        
                        for sentiment in ['positive', 'negative']:
                            try:
                                news = filter_by_percentile_and_label(valid_news, sentiment, 20)
                                if not news.empty:
                                    news_summary[sentiment.capitalize()] = list(news['news_title'])
                            except Exception:
                                continue
            except Exception as e:
                print(f"뉴스 처리 중 오류 발생: {e}")
        
        return news_summary

    def _get_response_content(self, response: Dict) -> str:
        """GPT 응답에서 content 추출"""
        try:
            return response["choices"][0]["message"]["content"]
        except (KeyError, IndexError):
            return ""
        
    def save_final_report(self, final_response: Dict) -> None:
        """최종 포트폴리오 매니저 보고서 저장"""
        page_title = f"{self.year}_{self.quarter}_analyst_rp"
        content = self._get_response_content(final_response)
        
        print(f"{page_title} 보고서를 노션 DB에 저장합니다...")
        to_DB('t_1', page_title, f"{self.quarter}_{self.year}", content)

In [4]:
import json

# JSON 파일 경로
json_file_path = "/Users/gamjawon/finTF/pipeline/notion_page_ids.json"

# JSON 파일 읽기 함수
def read_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        return {}
    
# JSON 파일 수정
def write_json(file_path, new_data):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(new_data, file, ensure_ascii=False, indent=4)
        
# 'pf_selection_agent' 섹션에서 종목 코드 추출
def get_tickers_from_json(agent_type, title):
   data = read_json(json_file_path)
   if agent_type in data and title in data[agent_type]:
       page_id = data[agent_type][title]
       content = get_all_text_from_page(page_id)
       
       try:
           # final_portfolio 부분 추출
           start = content.find("'final_portfolio'")
           end = content.find("'corp_analysis_report'")
           portfolio_str = content[start:end].strip()
           
           # 종목코드만 추출
           import re
           tickers = re.findall(r"'(\d{6})'", portfolio_str)
           return list(set(tickers))  # 중복 제거
           
       except Exception as e:
           print(f"Error: {e}")
           return []
   return []

# 예시로 'pf_selection_agent'의 '2022_Q4_init_pf'에서 종목 코드 추출
if __name__ == "__main__":
    # 'pf_selection_agent'에서 '2022_Q4_init_pf' 종목 코드 가져오기
    tickers = get_tickers_from_json('pf_selection_agent', '2022_Q4_init_pf')

    # 연도 및 분기 설정
    year = "2022"
    quarter = "Q4"

    # 분석기 객체 생성
    analyzer = InvestmentReportGenerator(tickers, year, quarter)

    # 분석 실행
    analyzer.analyze_stocks()
    
    # 개별 보고서 생성 
    for ticker in tickers:
        analyzer.generate_individual_report(ticker)



UK에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/gamjawon/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/UK/Inflation Rate/2022/2022_Inflation Rate.csv'

=== 093050 분석 중... ===

=== 035720 분석 중... ===
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/gamjawon/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/035720/_035720_재무제표 ().csv'
035720의 이전 년도 재무 데이터를 불러올 수 없습니다.
035720의 fin_statement_info 정보를 확인할 수 없습니다.

=== 031430 분석 중... ===

=== 011070 분석 중... ===

=== 000480 분석 중... ===

=== 039130 분석 중... ===

=== 033660 분석 중... ===
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/gamjawon/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/033660/_033660_재무제표 ().csv'
/Users/gamjawon/finTF/pipeline/sub_func/get_info/../../../store_data/raw/market_data/price/033660/2022.10/2022.10_033660.csv 파일을 찾을 수 없습니다.
/Users

/var/folders/y7/7hr2z6vd3fb6nbtgn0gwlm840000gn/T/ipykernel_36759/528922390.py:304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 031430 분석 중... ===


/var/folders/y7/7hr2z6vd3fb6nbtgn0gwlm840000gn/T/ipykernel_36759/528922390.py:304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 011070 분석 중... ===


...

/var/folders/y7/7hr2z6vd3fb6nbtgn0gwlm840000gn/T/ipykernel_36759/528922390.py:304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 011420 분석 중... ===
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/gamjawon/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/011420/_011420_재무제표 ().csv'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/gamjawon/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/011420/_011420_재무제표 ().csv'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/gamjawon/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/011420/_011420_재무제표 ().csv'
011420의 이전 년도 재무 데이터를 불러올 수 없습니다.
011420의 fin_statement_info 정보를 확인할 수 없습니다.

=== 025560 분석 중... ===

=== 000990 분석 중... ===


/var/folders/y7/7hr2z6vd3fb6nbtgn0gwlm840000gn/T/ipykernel_36759/528922390.py:304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 020000 분석 중... ===
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/gamjawon/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/020000/_020000_재무제표 ().csv'
020000의 이전 년도 재무 데이터를 불러올 수 없습니다.
020000의 fin_statement_info 정보를 확인할 수 없습니다.

=== 023800 분석 중... ===

=== 003160 분석 중... ===
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/gamjawon/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/003160/_003160_재무제표 ().csv'
003160의 이전 년도 재무 데이터를 불러올 수 없습니다.
003160의 fin_statement_info 정보를 확인할 수 없습니다.


/var/folders/y7/7hr2z6vd3fb6nbtgn0gwlm840000gn/T/ipykernel_36759/528922390.py:304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:


In [5]:
if __name__ == "__main__":
    # 최종 보고서 생성
    final_report = analyzer.generate_final_report()

    # 노션에 저장
    analyzer.save_final_report(final_report)

2022_Q4_analyst_rp 보고서를 노션 DB에 저장합니다...
페이지 생성 완료: 184cd049-9633-8185-81a8-cde6fce94a1e
텍스트 블럭 추가 완료


In [6]:
print(final_report)

{'id': 'chatcmpl-AsrWPJxxuhuvJkaEOjQapgojtmKw8', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 최종 투자전략 보고서\n\n## 1. 거시경제 분석 요약\n\n### 글로벌 동향 핵심 포인트\n- 금융업과 증권업 전반에서 주가 상승세가 뚜렷하며, 디지털화 및 기술 혁신이 진행되고 있음.\n- 미국 연준의 금리 인상 지속 여부와 글로벌 경제 불확실성이 주요 투자 변수로 작용하고 있음.\n- 산업 전반에서 ESG(환경, 사회, 지배구조) 경영에 대한 관심이 높아지고 있으며, 지속 가능한 성장 가능성이 주목받고 있음.\n\n### 주요 리스크 요인\n- 글로벌 경제 둔화 및 인플레이션 지속 가능성.\n- 금융 시장의 변동성이 클 경우, 투자자의 심리에 부정적인 영향을 미칠 수 있음.\n- 각국의 규제 변화에 따른 사업 운영의 불확실성.\n\n## 2. 개별 종목 분석 종합\n\n### 각 종목 투자매력도 비교\n| 종목 코드 | 회사명 | PER | PBR | ROE | 이익 성장률 | 목표가 | 투자포인트 |\n|-----------|--------|-----|-----|-----|-------------|--------|------------|\n| 093050    | A사    | 2.62| 0.28| 10.70%| 30.22%     | 20,000 | 재무 건전성, 성장성 |\n| 035720    | B사    | 7.44| 1.06| 14.25%| -          | 65,000 | 클라우드 성장성 |\n| 031430    | C사    | 7.44| 1.06| 14.25%| 43.77%     | 30,000 | 저평가 상태 |\n| 011070    | D사    | 6.10| 1.40| 22.97%| 10.31%     | 300,000| 안정적 성장 |\n| 000480    | E

In [7]:
class PortfolioManagerReportGenerator:
    def __init__(self, tickers: list, year: str, quarter: str):
        self.tickers = tickers
        self.year = year
        self.quarter = quarter
        self.prompts = self._initialize_prompts()
        self.responses = {}
        self.report_data = {}
        self.analyst_report = None
        self.current_portfolio = None
        self.individual_reports = {}
        
    def _initialize_prompts(self) -> Dict[str, str]:
        return {
            "individual_portfolio_system": """당신은 자산운용사의 포트폴리오 매니저입니다.
해당 종목에 대한 애널리스트 리서치 보고서를 검토하여 포트폴리오 운용 전략을 제시하세요.

# 1. 종목 현황
- 현재 비중과 추이
- 주요 위험/수익 지표
- 투자 성과 분석

# 2. 투자 전략
- 적정 비중과 근거
- 핵심 매력도/리스크
- 비중 조정 방향

# 3. 리스크 관리
- 손절/이익실현 기준
- 주요 모니터링 지표

응답은 markdown 형식으로 작성""",

            "final_portfolio_system": """당신은 자산운용사의 수석 포트폴리오 매니저입니다.
개별 종목 포트폴리오 보고서들을 종합하여 전체 포트폴리오 최종 운용 전략을 제시하세요.

# 1. 포트폴리오 종합 현황
- 전체 구성과 섹터 비중
- 종목별 성과 비교
- 핵심 위험/수익 특성

# 2. 전략적 자산배분
- 섹터별 비중 전략
- 종목간 상대매력도
- 전체 위험분산 방안

# 3. 최종 포트폴리오 조정안
- 종목별 비중 조정 방향
- 편입/편출 검토
- 우선순위와 실행계획

# 4. 종합 리스크 관리
- 포트폴리오 전체 관점
- 개별종목 리스크 통합 관리
- 주요 모니터링 지표

작성 지침:
- 개별 보고서들의 분석을 통합하여 결론 도출
- 종목간 상대가치 고려한 전략 수립
- 구체적 실행방안 제시

응답은 markdown 형식으로 작성""",
            "individual_portfolio_prompt": "",
            "final_portfolio_prompt": ""
        }
        
    def generate_individual_report(self, ticker: str, analyst_report: str, portfolio: Dict) -> str:
        """개별 종목 포트폴리오 보고서 생성"""
        prompt = f"종목코드: {ticker}\n"
        prompt += f"현재 포트폴리오 구성: {portfolio}\n"
        prompt += f"애널리스트 보고서: {analyst_report}"
        
        response = to_GPT(self.prompts["individual_portfolio_system"], prompt)
        self.individual_reports[ticker] = response
        return response
        
    def set_analyst_report(self, report: str) -> None:
        """애널리스트 보고서 설정"""
        self.analyst_report = report
        
    def set_current_portfolio(self, portfolio: Dict) -> None:
        """현재 포트폴리오 설정"""
        self.current_portfolio = portfolio
        
    def generate_final_report(self) -> str:
        """최종 포트폴리오 매니저 보고서 생성"""
        if not self.individual_reports:
            raise ValueError("Individual reports must be generated first")
        
        combined_prompt = []
        for ticker in self.tickers:
            if ticker in self.individual_reports:
                combined_prompt.append(f"\n=== {ticker} 포트폴리오 보고서 ===")
                combined_prompt.append(self._get_response_content(self.individual_reports[ticker]))
        
        self.prompts["final_portfolio_prompt"] = "\n".join(combined_prompt)
        final_response = to_GPT(self.prompts["final_portfolio_system"], 
                              self.prompts["final_portfolio_prompt"])
        return final_response
    
    def _get_response_content(self, response: Dict) -> str:
        """GPT 응답에서 content 추출"""
        try:
            return response["choices"][0]["message"]["content"]
        except (KeyError, IndexError):
            return ""
        
    def save_final_report(self, final_response: Dict) -> None:
        """최종 포트폴리오 매니저 보고서 저장"""
        page_title = f"{self.year}_{self.quarter}_final_portfolio_report"
        content = self._get_response_content(final_response)
        
        print(f"{page_title} 보고서를 노션 DB에 저장합니다...")
        to_DB('t_1', page_title, f"{self.quarter}_{self.year}", content)



In [8]:
def get_current_portfolio():
    data = read_json(json_file_path)
    agent_type = 'pf_selection_agent'
    title = '2022_Q4_init_pf'
    # 특정 agent_type과 title에 해당하는 종목 코드 추출
    if agent_type in data and title in data[agent_type]:
        portfolio = data[agent_type][title]
        return portfolio
    else:
        return []

In [9]:
if __name__ == "__main__":
# 2. 종합 포트폴리오 매니저 보고서 생성
    # 티커는 대표 종목으로 설정하거나 "MULTI"와 같은 식별자 사용
    portfolio_manager = PortfolioManagerReportGenerator(tickers, year, quarter)
    
    # 각 종목별 포트폴리오 보고서 생성
    for ticker in tickers:
        analyst_report = portfolio_manager.set_analyst_report(final_report["choices"][0]["message"]["content"])# 애널리스트 보고서 가져오기
        current_portfolio = portfolio_manager.set_current_portfolio(get_current_portfolio())# 현재 포트폴리오 정보 가져오기
        portfolio_manager.generate_individual_report(ticker, analyst_report, current_portfolio)
        print(f"{ticker} 포폴 매니저 보고서 추출 완료")
        
    # 보고서 생성
    portfolio_report = portfolio_manager.generate_final_report()
    
    # 보고서 출력
    print("\n=== 포트폴리오 매니저 종합 보고서 ===")
    print(portfolio_report["choices"][0]["message"]["content"])
    print("=== 보고서 끝 ===")
    
    # 선택적으로 노션에 저장
    portfolio_manager.save_final_report(portfolio_report)

093050 포폴 매니저 보고서 추출 완료
035720 포폴 매니저 보고서 추출 완료
031430 포폴 매니저 보고서 추출 완료
011070 포폴 매니저 보고서 추출 완료
000480 포폴 매니저 보고서 추출 완료
039130 포폴 매니저 보고서 추출 완료
033660 포폴 매니저 보고서 추출 완료
195870 포폴 매니저 보고서 추출 완료
029460 포폴 매니저 보고서 추출 완료
264900 포폴 매니저 보고서 추출 완료
077500 포폴 매니저 보고서 추출 완료
267850 포폴 매니저 보고서 추출 완료
092220 포폴 매니저 보고서 추출 완료
004370 포폴 매니저 보고서 추출 완료
001740 포폴 매니저 보고서 추출 완료
007810 포폴 매니저 보고서 추출 완료
008770 포폴 매니저 보고서 추출 완료
011420 포폴 매니저 보고서 추출 완료
025560 포폴 매니저 보고서 추출 완료
000990 포폴 매니저 보고서 추출 완료
020000 포폴 매니저 보고서 추출 완료
023800 포폴 매니저 보고서 추출 완료
003160 포폴 매니저 보고서 추출 완료

=== 포트폴리오 매니저 종합 보고서 ===
# 포트폴리오 최종 운용 전략 보고서

## 1. 포트폴리오 종합 현황
### 전체 구성과 섹터 비중
현재 포트폴리오는 다양한 종목으로 구성되어 있으며, 각 종목의 비중은 다음과 같습니다:
- **종목 1 (093050)**: 5%
- **종목 2 (035720)**: 5%
- **종목 3 (031430)**: 5%
- **종목 4 (011070)**: 5%
- **종목 5 (000480)**: 5%
- **종목 6 (039130)**: 5%
- **종목 7 (033660)**: 5%
- **종목 8 (195870)**: 5%
- **종목 9 (029460)**: 5%
- **종목 10 (264900)**: 5%
- **종목 11 (077500)**: 5%
- **종목 12 (267850)**: 5%
- **종목 13 (092220)**: 5%


In [10]:
class TraderReportGenerator:
    def __init__(self, tickers: list, year: str, quarter: str):
        self.tickers = tickers
        self.year = year
        self.quarter = quarter
        self.prompts = self._initialize_prompts()
        self.individual_reports = {}
        self.responses = {ticker: {} for ticker in tickers}
        self.report_data = {}
        self.start_date, self.end_date = self._get_date_range()
        self.price_data = {}
        self.analyst_reports = {}
        self.pm_reports = {}
        self.price_predictions = {}
        
    def _initialize_prompts(self) -> Dict[str, str]:
        """프롬프트 초기화"""
        return {
            "individual_trader_system": """당신은 증권사의 트레이더입니다. 해당 종목의 데이터와 보고서를 분석하여 구체적인 매매 전략을 제시하세요.

# 종목 기본 분석
- 현재가 동향과 기술적 신호
- 예측가격 분석
- 거래량 특징

# 매매 전략
- 매매 방향과 근거
- 진입/청산 가격대
- 리스크 관리 전략

모든 분석은 제공된 데이터에 기반하여 작성하세요.
응답은 markdown 형식으로 작성""",

            "final_trader_system": """당신은 증권사의 수석 트레이더입니다. 개별 종목 트레이딩 보고서들을 종합하여 최종 매매 전략을 제시하세요.

# 시장 종합 분석
- 주요 매매 환경
- 전반적 매매 전략

# 우선 매매 종목
- 상위 5-7개 종목 선정과 근거
- 구체적 매매 전략
- 핵심 리스크 관리

# 기타 종목 전략
- 매수/매도 종목 구분
- 주요 진입/청산 전략

작성 지침:
- 개별 보고서 분석 통합
- 우선순위 기반 전략 수립
- 구체적 실행 방안 제시

응답은 markdown 형식으로 작성""",
            "individual_trader_prompt": "",
            "final_trader_prompt": ""
        }

    def _get_date_range(self) -> tuple:
        """분기에 해당하는 시작일과 종료일 반환"""
        quarter_months = {
            'Q1': ('01', '03'),
            'Q2': ('04', '06'),
            'Q3': ('07', '09'),
            'Q4': ('10', '12')
        }
        
        if self.quarter in quarter_months:
            start_month, end_month = quarter_months[self.quarter]
            start_date = f"{self.year}{start_month}01"
            end_date = f"{self.year}{end_month}{'30' if end_month in ['06', '09'] else '31'}"
            return start_date, end_date
        else:
            raise ValueError(f"Invalid quarter: {self.quarter}")

    def set_price_data(self) -> None:
        """주가 데이터 설정"""
        try:
            for ticker in self.tickers:
                self.price_data[ticker] = stock_price_info(ticker, self.start_date, self.end_date)
        except Exception as e:
            print(f"가격 데이터 설정 중 오류 발생: {str(e)}")


    def set_analyst_report(self, report: str) -> None:
        """애널리스트 보고서 설정"""
        self.analyst_report = report

    def set_pm_report(self, report: str) -> None:
        """포트폴리오 매니저 보고서 설정"""
        self.pm_report = report

    def get_price_prediction(self) -> None:
        """GRU 모델을 사용한 가격 예측"""
        try:
            self.price_predictions = predict_multiple_prices(
                self.tickers,
                self.start_date,
                self.end_date
            )
        except Exception as e:
            print(f"가격 예측 모델 실행 중 오류 발생: {str(e)}")
    
    def generate_individual_report(self, ticker: str) -> str:
        """개별 종목 트레이더 보고서 생성"""
        price_data = self.price_data.get(ticker, {})
        if hasattr(price_data, 'to_dict'):
            price_data = price_data.to_dict()
            
        prompt = f"종목코드: {ticker}\n"
        prompt += f"가격 데이터: {price_data}\n"
        prompt += f"가격 예측: {self.price_predictions.get(ticker, {})}\n"
        prompt += f"애널리스트 보고서: {self.analyst_reports.get(ticker, '정보 없음')}\n"
        prompt += f"PM 보고서: {self.pm_reports.get(ticker, '정보 없음')}"

        response = to_GPT(self.prompts["individual_trader_system"], prompt)
        self.individual_reports[ticker] = response
        return response

    def generate_final_report(self) -> str:
        """최종 트레이더 보고서 생성"""
        if not self.individual_reports:
            raise ValueError("Individual reports must be generated first")
        
        combined_prompt = []
        for ticker in self.tickers:
            if ticker in self.individual_reports:
                combined_prompt.append(f"\n=== {ticker} 트레이딩 보고서 ===")
                combined_prompt.append(self._get_response_content(self.individual_reports[ticker]))
        
        self.prompts["final_trader_prompt"] = "\n".join(combined_prompt)
        final_response = to_GPT(self.prompts["final_trader_system"], 
                              self.prompts["final_trader_prompt"])
        return final_response
    
    def _get_response_content(self, response: Dict) -> str:
        try:
            return response["choices"][0]["message"]["content"]
        except (KeyError, IndexError):
            return ""

    def save_final_report(self, final_response: Dict) -> None:
        page_title = f"{self.year}_{self.quarter}_final_trader_report"
        content = self._get_response_content(final_response)
        
        print(f"{page_title} 보고서를 노션 DB에 저장합니다...")
        to_DB('t_1', page_title, f"{self.quarter}_{self.year}", content)

    

In [12]:
if __name__ == "__main__":
    tickers = get_tickers_from_json('pf_selection_agent', '2022_Q4_init_pf')
    year = "2022"
    quarter = "Q4"
    
    trader = TraderReportGenerator(tickers, year, quarter)
    
    # 필요한 데이터 설정
    trader.set_price_data()
    trader.get_price_prediction()
    
    # 각 종목별 트레이더 보고서 생성
    for ticker in tickers:
        trader.generate_individual_report(ticker)
        print(f"{ticker} 트레이더 보고서 추출 완료")
    
    # 최종 트레이더 보고서 생성
    final_report = trader.generate_final_report()
    trader.save_final_report(final_report)

로드 데이터 함수 시작
[DEBUG] 새로운 데이터 로드 시도
[DEBUG] 연도: 2022, 월: 10
[DEBUG] 매핑된 분기: Q4
[DEBUG] 재무제표 데이터 로드: True

[DEBUG] 외국인 보유 비중 데이터 로드 시도
[DEBUG] 외국인 보유 비중 데이터 로드 성공
[DEBUG] 데이터 병합 완료
[DEBUG] 최종 데이터 shape: (62, 5)
[DEBUG] 학습 데이터 통계:
X_train shape: (47, 15, 3)
y_train shape: (47,)
X_train 값 범위: 0.0 ~ 1.0
y_train 값 범위: 0.04444444444444429 ~ 1.0
Epoch 1/50


/Users/gamjawon/finTF/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 0.1748 - mae: 0.5197 - val_loss: 0.2542 - val_mae: 0.7091 - learning_rate: 0.0010
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1506 - mae: 0.4792 - val_loss: 0.2092 - val_mae: 0.6421 - learning_rate: 0.0010
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0980 - mae: 0.3712 - val_loss: 0.1735 - val_mae: 0.5839 - learning_rate: 0.0010
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0925 - mae: 0.3646 - val_loss: 0.1439 - val_mae: 0.5308 - learning_rate: 0.0010
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0712 - mae: 0.3170 - val_loss: 0.1210 - val_mae: 0.4854 - learning_rate: 0.0010
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0587 - mae: 0.2866 - val_loss: 0.1003 - val_mae: 0.4404 - learning_rate: 0.0010
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0514 - mae: 0.2694 - val_loss: 0.0814 - val_mae: 0.3949 - learning_rate: 0.0010
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s

/Users/gamjawon/finTF/venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/gamjawon/finTF/venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


[DEBUG] 외국인 보유 비중 데이터 로드 성공
[DEBUG] 데이터 병합 완료
[DEBUG] 최종 데이터 shape: (62, 5)
[DEBUG] 학습 데이터 통계:
X_train shape: (47, 15, 3)
y_train shape: (47,)
X_train 값 범위: 0.0 ~ 1.0
y_train 값 범위: 0.0 ~ 0.8571428571428577
Epoch 1/50


...

[DEBUG] 외국인 보유 비중 데이터 로드 성공
[DEBUG] 데이터 병합 완료
[DEBUG] 최종 데이터 shape: (62, 5)
[DEBUG] 학습 데이터 통계:
X_train shape: (47, 15, 3)
y_train shape: (47,)
X_train 값 범위: 0.0 ~ 1.0000000000000002
y_train 값 범위: 0.0 ~ 0.4136253041362532
Epoch 1/50


/Users/gamjawon/finTF/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 0.0638 - mae: 0.3504 - val_loss: 0.0095 - val_mae: 0.1190 - learning_rate: 0.0010
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0503 - mae: 0.3077 - val_loss: 0.0038 - val_mae: 0.0677 - learning_rate: 0.0010
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0320 - mae: 0.2422 - val_loss: 0.0018 - val_mae: 0.0491 - learning_rate: 0.0010
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0187 - mae: 0.1801 - val_loss: 0.0029 - val_mae: 0.0667 - learning_rate: 0.0010
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0106 - mae: 0.1271 - val_loss: 0.0063 - val_mae: 0.0991 - learning_rate: 0.0010
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0058 - mae: 0.0881 - val_loss: 0.0120 - val_mae: 0.1469 - learning_rate: 0.0010
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0043 - mae: 0.0779 - val_loss: 0.0187 - val_mae: 0.1878 - learning_rate: 0.0010
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s

/Users/gamjawon/finTF/venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/gamjawon/finTF/venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


093050 트레이더 보고서 추출 완료
035720 트레이더 보고서 추출 완료
031430 트레이더 보고서 추출 완료
011070 트레이더 보고서 추출 완료
000480 트레이더 보고서 추출 완료
039130 트레이더 보고서 추출 완료
033660 트레이더 보고서 추출 완료
195870 트레이더 보고서 추출 완료
029460 트레이더 보고서 추출 완료
264900 트레이더 보고서 추출 완료
077500 트레이더 보고서 추출 완료
267850 트레이더 보고서 추출 완료
092220 트레이더 보고서 추출 완료
004370 트레이더 보고서 추출 완료
001740 트레이더 보고서 추출 완료
007810 트레이더 보고서 추출 완료
008770 트레이더 보고서 추출 완료
011420 트레이더 보고서 추출 완료
025560 트레이더 보고서 추출 완료
000990 트레이더 보고서 추출 완료
020000 트레이더 보고서 추출 완료
023800 트레이더 보고서 추출 완료
003160 트레이더 보고서 추출 완료
2022_Q4_final_trader_report 보고서를 노션 DB에 저장합니다...
페이지 생성 완료: 184cd049-9633-81be-b80f-e1c6cc9e6e23
텍스트 블럭 추가 완료


In [ ]:
fin_data = fin_statement_info("005930", "2020", "Q4")
print("재무제표 데이터:")
print(fin_data)

재무제표 데이터:
   Stock Price        PER       PBR       ROE  Profit Growth Rate (%)  \
0        81000  18.310946  1.752331  0.095699                21.47751   

   CAGR (%)  
0  0.214775  


In [ ]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.6 MB/s eta 0:00:00
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 24.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 33.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
import pandas as pd
import numpy as np
from sub_func import *

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import os
import joblib
from pykrx import stock

def predict_multiple_prices(tickers: list, start_date: str, end_date: str, price_data_dict=None) -> dict:
    predictions = {}
    
    if not os.path.exists('models'):
        os.makedirs('models')
    
    for ticker in tickers:
        try:
            data = load_stock_data(ticker, start_date, end_date)
            if data is None:
                continue
                
            # PER이 0인 경우 처리
            if np.all(data['PER'] == 0):
                print("[WARNING] PER이 모두 0입니다. 평균값으로 대체합니다.")
                data['PER'] = 15.0  # 일반적인 PER 평균값으로 대체
            
            # 스케일링
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data[['close', 'high', 'PER', 'foreign_holding']])
            data[['close', 'high', 'PER', 'foreign_holding']] = scaled_data
            
            # 시계열 데이터 준비
            window_size = 15
            X = []
            y = []
            for i in range(window_size, len(data)):
                X.append(data[['high', 'PER', 'foreign_holding']].values[i-window_size:i])
                y.append(data['close'].values[i])
            X = np.array(X)
            y = np.array(y)
            
            # 모델 학습
            model = create_and_train_model(X, y, ticker)
            
            # 예측 수행
            last_sequence = X[-1:]
            future_predictions = []
            current_sequence = last_sequence.copy()
            
            for _ in range(20):
                pred = model.predict(current_sequence, verbose=0)
                future_predictions.append(float(pred[0, 0]))
                
                current_sequence = np.roll(current_sequence, -1, axis=1)
                current_sequence[0, -1] = [pred[0, 0], data['PER'].iloc[-1], data['foreign_holding'].iloc[-1]]
            
            # 예측값 역변환
            future_predictions = np.array(future_predictions).reshape(-1, 1)
            future_predictions = np.concatenate([future_predictions, np.zeros((len(future_predictions), 3))], axis=1)
            future_predictions = scaler.inverse_transform(future_predictions)[:, 0]
            
            predictions[ticker] = {
                'current_price': float(scaler.inverse_transform([[data['close'].iloc[-1], 0, 0, 0]])[0, 0]),
                'predicted_prices': future_predictions.tolist(),
                'prediction_dates': pd.date_range(
                    start=pd.to_datetime(data['date'].iloc[-1]) + pd.Timedelta(days=1),
                    periods=20
                ).strftime('%Y-%m-%d').tolist()
            }
            
        except Exception as e:
            print(f"{ticker} 예측 중 오류 발생: {str(e)}")
            predictions[ticker] = None
    
    return predictions

def predict_price(ticker: str, start_date: str = None, end_date: str = None) -> dict:
    """
    GRU 모델을 사용하여 주가를 예측하는 함수
    
    Args:
        ticker (str): 주식 종목 코드
        start_date (str): 예측 시작일 (YYYYMMDD 형식)
        end_date (str): 예측 종료일 (YYYYMMDD 형식)
    
    Returns:
        dict: 예측 결과를 담은 딕셔너리
    """
    try:
        # 모델 및 스케일러 경로 설정
        MODEL_PATH = f'models/{ticker}_gru_model.h5'
        SCALER_PATH = f'models/{ticker}_scaler.pkl'
        
        # 데이터 로드 
        data = load_stock_data(ticker, start_date, end_date)  
        
        # 데이터 전처리
        data['date'] = pd.to_datetime(data['date'])
        data = data.sort_values('date')
        
        # 저장된 스케일러 로드 또는 새로 생성
        if os.path.exists(SCALER_PATH):
            scaler = joblib.load(SCALER_PATH)
        else:
            scaler = MinMaxScaler()
            data[['close', 'high', 'PER', 'foreign_holding']] = scaler.fit_transform(
                data[['close', 'high', 'PER', 'foreign_holding']]
            )
            joblib.dump(scaler, SCALER_PATH)
        
        # 시계열 데이터 준비
        window_size = 15
        X = []
        for i in range(window_size, len(data)):
            X.append(data[['high', 'PER', 'foreign_holding']].values[i-window_size:i])
        X = np.array(X)
        
        # 모델 로드 또는 새로 생성
        if os.path.exists(MODEL_PATH):
            model = load_model(MODEL_PATH)
        else:
            model = create_and_train_model(X, data['close'].values[window_size:], ticker)
        
        # 다음 분기 예측
        future_predictions = []
        last_sequence = X[-1:]
        
        # 다음 20일(약 한 달) 예측
        for _ in range(20):
            next_pred = model.predict(last_sequence)
            future_predictions.append(next_pred[0, 0])
            
            # 다음 예측을 위한 시퀀스 업데이트
            last_sequence = np.roll(last_sequence, -1, axis=1)
            last_sequence[0, -1] = next_pred
        
        # 예측값 역변환
        future_predictions = np.array(future_predictions).reshape(-1, 1)
        future_predictions = scaler.inverse_transform(
            np.concatenate((future_predictions, np.zeros((future_predictions.shape[0], 3))), axis=1)
        )[:, 0]
        
        # 결과 정리
        result = {
            'current_price': data['close'].iloc[-1],
            'predicted_prices': future_predictions.tolist(),
            'prediction_dates': pd.date_range(
                start=data['date'].iloc[-1] + pd.Timedelta(days=1), 
                periods=20
            ).strftime('%Y-%m-%d').tolist(),
            'confidence_level': calculate_confidence_level(model, X, data['close'].values[window_size:])
        }
        
        return result
        
    except Exception as e:
        print(f"예측 중 오류 발생: {str(e)}")
        return None

def create_and_train_model(X_train, y_train, ticker):
    """GRU 모델 생성 및 학습"""
    print(f"[DEBUG] 학습 데이터 통계:")
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_train 값 범위: {np.min(X_train)} ~ {np.max(X_train)}")
    print(f"y_train 값 범위: {np.min(y_train)} ~ {np.max(y_train)}")

    # 입력 데이터 검증
    if np.any(np.isnan(X_train)) or np.any(np.isinf(X_train)):
        raise ValueError("입력 데이터에 NaN 또는 무한값이 포함되어 있습니다.")

    if np.any(np.isnan(y_train)) or np.any(np.isinf(y_train)):
        raise ValueError("타겟 데이터에 NaN 또는 무한값이 포함되어 있습니다.")

    # 모델 구조
    model = Sequential([
        GRU(32, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    
    # 컴파일
    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss='huber',
        metrics=['mae']
    )
    
    # 콜백
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=0.0001
        )
    ]
    
    # 학습
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=16,
        validation_split=0.2,
        callbacks=callbacks,
        verbose=1
    )
    
    # 학습 결과 검증
    print("\n[DEBUG] 모델 평가:")
    val_predictions = model.predict(X_train[-int(len(X_train)*0.2):])
    val_true = y_train[-int(len(y_train)*0.2):]
    mse = np.mean((val_predictions - val_true) ** 2)
    print(f"검증 세트 MSE: {mse}")
    
    return model

def calculate_confidence_level(model, X, y_true):
    """예측 신뢰도 계산"""
    y_pred = model.predict(X)
    mse = np.mean((y_pred - y_true.reshape(-1, 1)) ** 2)
    confidence = np.exp(-mse)  # 0~1 사이의 값으로 변환
    return float(confidence)

def load_stock_data(ticker: str, start_date: str, end_date: str, price_data=None) -> pd.DataFrame:
    """
    주식 데이터를 로드하는 함수
    price_data: TraderReportGenerator에서 이미 로드된 가격 데이터
    """
    print("로드 데이터 함수 시작")
    try:
        if price_data is not None:
            print("[DEBUG] 기존 price_data 사용")  # 디버깅
            selected_data = pd.DataFrame()
            selected_data['close'] = price_data['Close']
            selected_data['high'] = price_data['High']
            selected_data['date'] = price_data.index
            selected_data = selected_data.reset_index(drop=True)
        else:
            print("[DEBUG] 새로운 데이터 로드 시도")  # 디버깅
            price_data = stock_price_info(ticker, start_date, end_date)
            
            if price_data is None:
                print(f"Warning: 가격 데이터를 가져올 수 없습니다: {ticker}")
                return None
            
            selected_data = pd.DataFrame()
            selected_data['close'] = price_data['Close']
            selected_data['high'] = price_data['High']
            selected_data['date'] = price_data.index
            selected_data = selected_data.reset_index(drop=True)
            
        # 연도와 분기 추출
        year = start_date[:4]
        month = start_date[4:6]
        print(f"[DEBUG] 연도: {year}, 월: {month}")  # 디버깅
        
        # 분기 매핑
        quarter_map = {
            'Q1': ['01', '02', '03'],
            'Q2': ['04', '05', '06'],
            'Q3': ['07', '08', '09'],
            'Q4': ['10', '11', '12']
        }
        
        quarter = next(q for q, months in quarter_map.items() if month in months)
        print(f"[DEBUG] 매핑된 분기: {quarter}")  # 디버깅
        
        # PER 데이터 가져오기
        fin_data = fin_statement_info(ticker, year, quarter)
        print(f"[DEBUG] 재무제표 데이터 로드: {fin_data is not None}")  # 디버깅
        
        if fin_data is not None:
            per_value = fin_data['PER'].iloc[0]
        else:
            per_value = None
            
        # PER 컬럼 추가
        selected_data['PER'] = per_value
        
        # 외국인 보유 비중 추가
        try:
            print("\n[DEBUG] 외국인 보유 비중 데이터 로드 시도")
            foreign_data = stock.get_exhaustion_rates_of_foreign_investment(start_date, end_date, ticker)
            print(f"[DEBUG] 외국인 보유 비중 데이터 로드 성공")
            
            # 데이터 병합을 위해 인덱스 처리
            selected_data['date'] = pd.to_datetime(selected_data['date'])
            foreign_data = foreign_data.reset_index()
            foreign_data.columns = ['date' if col == '날짜' else col for col in foreign_data.columns]
            foreign_data['date'] = pd.to_datetime(foreign_data['date'])
            
            # 날짜 기준으로 데이터 병합
            selected_data = pd.merge(selected_data, 
                                   foreign_data[['date', '지분율']], 
                                   on='date', 
                                   how='left')
            selected_data = selected_data.rename(columns={'지분율': 'foreign_holding'})
            
            print("[DEBUG] 데이터 병합 완료")
        except Exception as e:
            print(f"[DEBUG] 외국인 보유 비중 로드 실패: {str(e)}")
            selected_data['foreign_holding'] = 0
            
        print(f"[DEBUG] 최종 데이터 shape: {selected_data.shape}")  # 디버깅
        return selected_data
        
    except Exception as e:
        print(f"데이터 로드 중 오류 발생: {str(e)}")
        print(f"[DEBUG] 오류 발생 위치 정보: {e.__traceback__.tb_lineno}")  # 디버깅
        return None